# Convolutional Neural Networks

This tutorial demonstrates training a simple Convolutional Neural Network (CNN) to classify MNIST digits. This network will achieve over 99% accuracy on the MNIST test set. Because this tutorial uses the Keras Sequential API, creating and training our model will take just a few lines of code.

## Import TensorFlow

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

  Using cached https://files.pythonhosted.org/packages/35/6b/11530768cac581a12952a2aad00e1526b89d242d0b9f59534ef6e6a1752f/astor-0.7.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8c/96/adbd4eafe72ce9b5ca6f168fbf109386e1b601f7c59926a11e9d7b7a5b44/google_pasta-0.1.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/90/85/64c82949765cfb246bbdaf5aca2d55f400f792655927a017710a78445def/Keras_Applications-1.0.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3a/3c/515afabfe4f29bfc0a67037efaf518c33d0076b32d22ba865241cee295c4/numpy-1.16.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/c0/bf/0315ef6a9fd3fc2346e85b0ff1f5f83ca17073f2c31ac719ab2e4da0d4a3/Keras_Preprocessing-1.0.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9c/50/1fd6e3cb2cc0b9f88aa396020b0a831a2e8fb63a4479d1dfb4ae64b654bf/grpcio-1.19.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.

## Download and prepare the MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Create the convolutional base

The code below defines the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument input_shape to our first layer.

In [3]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let display the architecture of our model so far.

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, we can afford (computationally) to add more output channels in each Conv2D layer.

## Add Dense layers on top

To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [5]:
model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Here's the complete architecture of our model.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

As you can see, our (3, 3, 64) outputs were flattened into vectors of shape (576) before going through two Dense layers.

## Compile and train the model

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=20)

Epoch 1/20
60000/60000 [==============================] - 9s 150us/sample - loss: 0.0175 - accuracy: 0.9946
Epoch 2/20
60000/60000 [==============================] - 9s 146us/sample - loss: 0.0138 - accuracy: 0.9957
Epoch 3/20
60000/60000 [==============================] - 9s 146us/sample - loss: 0.0115 - accuracy: 0.9964
Epoch 4/20
60000/60000 [==============================] - 9s 143us/sample - loss: 0.0101 - accuracy: 0.9966
Epoch 5/20
60000/60000 [==============================] - 9s 147us/sample - loss: 0.0088 - accuracy: 0.9970
Epoch 6/20
60000/60000 [==============================] - 9s 143us/sample - loss: 0.0078 - accuracy: 0.9977
Epoch 7/20
60000/60000 [==============================] - 9s 142us/sample - loss: 0.0073 - accuracy: 0.9978
Epoch 8/20
60000/60000 [==============================] - 9s 146us/sample - loss: 0.0066 - accuracy: 0.9979
Epoch 9/20
60000/60000 [==============================] - 9s 150us/sample - loss: 0.0080 - accuracy: 0.9974
Epoch 10/20
60000/60000 [===

## Evaluate the model

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print(test_acc)

10000/10000 [==============================] - 1s 72us/sample - loss: 0.0585 - accuracy: 0.9930
0.993
